In [8]:
import requests
import json
import pandas as pd
from datetime import datetime

from datetime import datetime, timedelta


longitude = str(-56.924304180864745)
latitude = str(-21.83960333376933)

start = "2022-12-15" # yyyy-mm-dd
end = "2023-12-25"  # yyyy-mm-dd

start_date = datetime.strptime(start, "%Y-%m-%d")
end_date = datetime.strptime(end, "%Y-%m-%d")

new_start = start_date.replace(day=1).strftime("%Y%m%d")
new_end = (end_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
new_end = new_end.strftime("%Y%m%d")
print(new_start, new_end)


base_url = (f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=PRECTOTCORR&community=RE&longitude={longitude}&latitude={latitude}&start={new_start}&end={new_end}&format=JSON")
api_request_url = base_url.format(longitude=longitude, latitude=latitude)
response = requests.get(url=api_request_url, verify=True, timeout=1000)
content = json.loads(response.content.decode('utf-8'))
df = pd.DataFrame.from_dict(content['properties']['parameter'])
df[df < 0] = 0
# Convert the index to datetime
#df.index = pd.to_datetime(df.index, format='%Y%m%d')

# Resample the data to monthly frequency and sum the values
# monthly_sum = df.resample('ME').sum()
# print(monthly_sum)
df

SyntaxError: invalid syntax (2854829086.py, line 31)

In [5]:
df.reset_index().rename(columns={'index': 'Date'})

,date,PRECTOTCORR
0,2022-12-01,2.47
1,2022-12-02,7.40
2,2022-12-03,0.95
3,2022-12-04,0.16
4,2022-12-05,0.00
...,...,...
391,2023-12-27,0.59
392,2023-12-28,0.00
393,2023-12-29,2.33
394,2023-12-30,12.03


In [25]:
df = pd.read_csv('NDVI_FJS_3_time_series.csv')

In [38]:
import plotly.graph_objects as go

fig = go.Figure()

# Add bar plot for monthly_sum
fig.add_trace(go.Bar(x=monthly_sum.index, y=monthly_sum['PRECTOTCORR'], name='Monthly Precipitation', yaxis='y2', marker_color='blue'))

# Add line plot for average_index
fig.add_trace(go.Scatter(x=df['date'], y=df['average_index'], mode='lines', name='Average Index', line=dict(color='green')))

# Update layout for dual y-axis
fig.update_layout(
    yaxis=dict(
        title='Average Index',
    ),
    yaxis2=dict(
        title='Precipitation (mm)',
        overlaying='y',
        side='right'
    ),
    xaxis=dict(title='Date'),
    title='Average Index and Monthly Precipitation'
)

fig.show()

In [39]:
import plotly.graph_objects as go

fig = go.Figure()

# Add bar plot for monthly_sum (behind the line plot)
fig.add_trace(go.Bar(x=monthly_sum.index, y=monthly_sum['PRECTOTCORR'], 
                     name='Monthly Precipitation', yaxis='y2', 
                     marker_color='blue', opacity=0.5))

# Add line plot for average_index (above the bar)
fig.add_trace(go.Scatter(x=df['date'], y=df['average_index'], 
                         mode='lines+markers', name='Average Index', 
                         line=dict(color='green', width=3)))

# Update layout for dual y-axis
fig.update_layout(
    yaxis=dict(
        title='Average Index',
    ),
    yaxis2=dict(
        title='Precipitation (mm)',
        overlaying='y',
        side='right'
    ),
    xaxis=dict(title='Date'),
    title=dict(
        text='Average Index and Monthly Precipitation',
        x=0.5,  # Center the title
        font=dict(size=20)
    ),
    barmode='overlay'
)

fig.show()


In [37]:
import plotly.graph_objects as go

fig = go.Figure()

# Add bar plot (set below the line explicitly)
fig.add_trace(go.Bar(
    x=monthly_sum.index, 
    y=monthly_sum['PRECTOTCORR'], 
    name='Monthly Precipitation', 
    yaxis='y2', 
    marker_color='blue'
))

# Add line plot (this goes on top)
fig.add_trace(go.Scatter(
    x=df['date'], 
    y=df['average_index'], 
    mode='lines+markers', 
    name='Average Index', 
    line=dict(color='green', width=3)
))

# Ensure correct layout and layering
fig.update_layout(
    yaxis=dict(
        title='Average Index',
        titlefont=dict(color='green'),
        tickfont=dict(color='green'),
        layer='above traces'  # Force the line plot layer to be on top
    ),
    yaxis2=dict(
        title='Precipitation (mm)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        overlaying='y',
        side='right',
        layer='below traces'  # Ensure bar plot is below the line
    ),
    xaxis=dict(title='Date'),
    title=dict(
        text='Average Index and Monthly Precipitation',
        x=0.5,  # Center the title
        font=dict(size=20)
    )
)

fig.show()
